In [1]:

using Revise
includet("../general_modules/channels.jl")
include("../general_modules/mesh.jl")
using .channels
using .mesh
using JSON

In [2]:
data = JSON.parsefile("test.json")
Systems_params= data["Systems"]
mesh_params= data["Mesh"]
fermion=false
α= α3b(fermion,0.0,0.0,1,Systems_params["lmax"],Systems_params["lmin"],Systems_params["λmax"],Systems_params["λmin"],Systems_params["s1"],Systems_params["s2"],Systems_params["s3"],Systems_params["t1"],Systems_params["t2"],Systems_params["t3"],Systems_params["MT"])

grid= initialmesh(mesh_params["nθ"],mesh_params["nx"],mesh_params["ny"],Float64(mesh_params["xmax"]),Float64(mesh_params["ymax"]),Float64(mesh_params["alpha"]))

For J=0.0 T=0.0 parity=1 Number of channels: 1
scaling factor for x: 0.3


Main.mesh.meshset(12, 12, 12, [-0.9815606342467192, -0.9041172563704748, -0.7699026741943047, -0.5873179542866175, -0.3678314989981802, -0.1252334085114689, 0.1252334085114689, 0.3678314989981802, 0.5873179542866175, 0.7699026741943047, 0.9041172563704748, 0.9815606342467192], [0.04717533638651175, 0.10693932599531826, 0.16007832854334625, 0.20316742672306587, 0.23349253653835492, 0.24914704581340288, 0.24914704581340288, 0.23349253653835492, 0.20316742672306587, 0.16007832854334625, 0.10693932599531826, 0.04717533638651175], [0.0848575044719742, 0.2856978124093846, 0.6049476415733046, 1.0477062209334076, 1.6216473060047178, 2.337844182123722, 3.2122166066967295, 4.268145709136995, 5.5415989902690015, 7.092535125720268, 9.036017587831902, 11.666785312828601], [0.14277897009800458, 0.259383321803658, 0.37995967777078926, 0.5068231073270444, 0.6428816610948332, 0.7921245262743847, 0.960438937417142, 1.1572465340299454, 1.3992307171257872, 1.7204236864089044, 2.2079451522012565, 3.2157606

In [ ]:
for i in 1:mesh_params["nx"]
    println(grid.xi[i], "   ", grid.dxi[i])
end

In [ ]:
using Revise
Revise.revise()
includet("Gcoefficient.jl")
using .Gcoefficient

initialY(λmax, lmax, nθ, nx, ny, cosθi, xi, yi, P::Char)
return Yλαout, Ylαin, Yλαin

Yλin[iθ, iy, ix, nch, perm_index]

In [ ]:
Gαα=computeGcoefficient(α, grid)

In [ ]:
using Revise
includet("twobody.jl")
using .twobodybound

In [ ]:
potname="MT"
bound_states=bound2b(grid, potname)

In [ ]:
using Revise
includet("spline.jl")
using .spline

In [ ]:
"""
Test the cubic Hermitian spline interpolation with a polynomial function.
"""
# Import Printf for formatting
using Printf
using Plots
function test_cubherm_interpolation()
    # Define the polynomial function f(x) = 5x^5 + 4x^4 + 3x^3 + 2x^2 + x
    f(x) = 5*x^5 + 4*x^4 + 3*x^3 + 2*x^2 + x
    
    # Create a set of old grid points (coarse grid)
    xold = collect(range(-1.0, 1.0, length=15))
    
    # Evaluate the function at the old grid points
    yold = f.(xold)
    
    # println("Original grid points:")
    # for i in 1:length(xold)
    #     println("x = $(xold[i]), f(x) = $(yold[i])")
    # end
    # println()
    
    # Create a set of test points (fine grid) where we want to interpolate
    xtest = collect(range(-0.95, 0.95, length=20))
    
    # Compute the exact values
    yexact = f.(xtest)
    
    # Interpolate using our cubic Hermitian spline
    yinterp = Float64[]
    for x in xtest
        push!(yinterp, cubherm_interp_point(xold, yold, x))
    end
    
    # Compute errors
    errors = abs.(yinterp - yexact)
    max_error = maximum(errors)
    avg_error = sum(errors) / length(errors)
    
    println("Interpolation results and errors:")
    println("   x       Exact        Interpolated       Error")
    println("------------------------------------------------------")
    for i in 1:length(xtest)
        # Use simple string formatting instead of @printf/@sprintf
        x_str = @sprintf("%.4f", xtest[i])
        exact_str = @sprintf("%.6f", yexact[i])
        interp_str = @sprintf("%.6f", yinterp[i])
        error_str = @sprintf("%.6e", errors[i])
        
        println("$(lpad(x_str, 7))  $(lpad(exact_str, 12))  $(lpad(interp_str, 12))  $(lpad(error_str, 12))")
    end
    
    println("\nMaximum error: $max_error")
    println("Average error: $avg_error")
    
    # Optional: Plot the results if Plots.jl is available
    try
        
        
        p = plot(xold, yold, seriestype=:scatter, 
             label="Original Points", markersize=6, legend=:topleft)
        
        plot!(p, xtest, yexact, 
              label="Exact Function", linewidth=2)
        
        plot!(p, xtest, yinterp, 
              label="Interpolated", linewidth=2, linestyle=:dash)
        
        # Plot error
        p_error = plot(xtest, errors, 
                   label="Interpolation Error", linewidth=2,
                   xlabel="x", ylabel="Error", title="Interpolation Error")
        
        # Display both plots
        plot(p, p_error, layout=(2,1), size=(800, 600),
             title=["Cubic Hermitian Spline Interpolation" ""])
        
        savefig("interpolation_test.png")
        println("\nPlot saved as 'interpolation_test.png'")
    catch
        println("\nPlotting skipped (Plots.jl not available)")
    end
    
    return max_error, avg_error
end

In [ ]:
using Printf 
test_cubherm_interpolation()

In [3]:
using Revise
includet("matrices.jl")
using .matrices


In [11]:
Rxy_matrix(α, grid)


Rxy_32[i, ip]=0.041862633701180894 + 0.0im
Rxy_32[i, ip]=0.14061295919547906 + 0.0im
Rxy_32[i, ip]=0.2939422814647594 + 0.0im
Rxy_32[i, ip]=0.4932333366821114 + 0.0im
Rxy_32[i, ip]=0.7252756142156462 + 0.0im
Rxy_32[i, ip]=0.9742601370693419 + 0.0im
Rxy_32[i, ip]=1.2232446599230375 + 0.0im
Rxy_32[i, ip]=1.4552869374565724 + 0.0im
Rxy_32[i, ip]=1.6545779926739244 + 0.0im
Rxy_32[i, ip]=1.8079073149432048 + 0.0im
Rxy_32[i, ip]=1.9066576404375029 + 0.0im
Rxy_32[i, ip]=1.9485202741386838 + 0.0im


144×144 Matrix{ComplexF64}:
    1.93121+0.0im   -0.0220442+0.0im  …   -5.44117e-8+0.0im
 -0.0174399+0.0im     0.117298+0.0im       4.30043e-8+0.0im
  0.0247987+0.0im    -0.016208+0.0im       2.48075e-7+0.0im
  0.0145382+0.0im   -0.0112598+0.0im       5.84556e-7+0.0im
 0.00954745+0.0im  -0.00618584+0.0im       1.13654e-6+0.0im
 0.00679503+0.0im  -0.00412472+0.0im  …    2.01629e-6+0.0im
 0.00501528+0.0im  -0.00294816+0.0im       3.39517e-6+0.0im
 0.00374958+0.0im  -0.00216214+0.0im       5.63293e-6+0.0im
 0.00283742+0.0im   -0.0016149+0.0im        9.7389e-6+0.0im
 0.00222369+0.0im  -0.00125352+0.0im       1.94193e-5+0.0im
           ⋮                          ⋱  
 -0.0531333+0.0im    0.0298267+0.0im  …  -0.000835024+0.0im
 -0.0374949+0.0im    0.0209503+0.0im     -0.000765099+0.0im
  0.0810505+0.0im   -0.0454954+0.0im       0.00127227+0.0im
   0.135052+0.0im   -0.0756464+0.0im        0.0029799+0.0im
  0.0516645+0.0im   -0.0286223+0.0im       0.00301525+0.0im
   -0.24529+0.0im     0.143041

In [8]:
using Revise
includet("threebodybound.jl")
using .threebodybound

In [9]:
ThreeBody_Bound(α, grid)

Eigenvalues: ComplexF64[-186.69975531527942 - 3.1262870996434495e-20im, -34.93563199814223 + 105.01178592840657im, -34.9356319981422 - 105.01178592840655im, -23.7963597424379 + 39.278809134660484im, -23.79635974243786 - 39.27880913466042im, -14.793798730210588 + 11.64264096791756im, -14.793798730210577 - 11.642640967917554im, -12.67575177807902 + 2.6995813269644602e-15im, -5.817288047371363 - 1.4523794475729652im, -5.817288047371363 + 1.4523794475729646im, -1.496659664354679 + 1.629732030371402im, -1.4966596643546775 - 1.6297320303714018im, -1.3879662665376493 - 0.40262796342504614im, -1.3879662665376493 + 0.40262796342504664im, -0.5647686839009148 - 1.9309007344983957e-16im, -0.26623115571987055 + 5.551115123125783e-17im, -0.15577972380193067 - 0.1665033655436612im, -0.15577972380193067 + 0.16650336554366127im, -0.12015995534275022 + 5.021652413150462e-18im, -0.030895320490459833 - 0.03770761652659625im, -0.030895320490459812 + 0.03770761652659633im, -0.004976599108435761 - 0.00453204

12-element Vector{Any}:
 (-186.69975531527942 - 3.1262870996434495e-20im, ComplexF64[-0.003755444708106131 + 4.5127902063555e-19im, 0.7965479021249625 + 0.0im, 0.1540701688624116 - 5.065691112520564e-18im, 0.0019261452567634803 + 6.671841118034141e-19im, 0.0006782255595794371 + 4.242140678569928e-19im, 0.0005277035088807319 + 4.839146010218334e-21im, 0.00013350592910266594 + 3.969175229237245e-20im, 5.806621957233223e-5 + 1.6789101372171338e-20im, 2.5204600374255415e-5 + 5.5019007553151984e-21im, 1.1735226131590289e-5 + 2.5637328442793062e-21im  …  -1.5334428340625855e-15 - 2.997728997218882e-31im, 1.1627624074997985e-14 + 3.27918784692793e-30im, 2.3287512412658796e-15 + 2.9038674411215135e-31im, -2.0375843308676297e-14 - 5.4647172871205994e-30im, -2.6325572810732104e-14 - 6.256221864463289e-30im, 1.1675754485962556e-14 + 2.520897404197782e-30im, 1.813149616639124e-13 + 5.457859120630447e-29im, -1.1442695158456378e-13 - 2.546489306243172e-29im, 3.896421173776847e-14 + 5.854858131663106

In [ ]:
T_matrix(α,grid) 

In [ ]:
V_matrix(α, grid, "MT")